In [12]:
import pandas as pd
import numpy as np
import datetime as dt
import pandas_datareader.data as web
import quandl
from tqdm import tqdm_notebook
import time
import io
import requests
%config IPCompleter.greedy=True

In [2]:
# Extract the list of S&P 500 companies from Wikipedia
symbols_table = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies", header=0)[0]

In [3]:
# Extract the list of symbols to use for iex
symbols = list(symbols_table['Symbol'])

#set the time frame to fetch iex stock data
start = dt.datetime(2013, 4, 20)
end = dt.datetime(2019, 4, 19)

iex_data = pd.DataFrame()
# Get stockdata using pandas_datareader and combine in a single dataframe
for stock in tqdm_notebook(symbols):
    # Initialise the timer
    time.sleep(0.25)
    # Extract stock data and stack it in a single dataframe
    temp_data = web.DataReader(stock, "iex", start, end)
    temp_data['symbol'] = stock
    iex_data = pd.concat([iex_data, temp_data])

# Save the data as a csv file for future use
iex_data.to_csv('iex_data.csv')

In [112]:
# Extract the list of symbols to use for tiingo
symbols = list(symbols_table['Symbol'])

#set the time frame to fetch tiingo stock data
start = dt.datetime(2009, 4, 19)
end = dt.datetime(2019, 4, 19)
apikey = 'd0d2f9e1d277fcb6a24269b60d736f72c928a223'

tiingo_data = pd.DataFrame()
# Get stockdata using pandas_datareader and combine in a single dataframe
for stock in tqdm_notebook(symbols):
    # Initialise the timer
    time.sleep(0.25)
    if '.' in stock:
        stock = stock.replace('.', '-')
    # Extract stock data and stack it in a single dataframe
    temp_data = web.get_data_tiingo(stock, start, end, api_key = apikey)
    temp_data.reset_index(inplace=True)
    tiingo_data = pd.concat([tiingo_data, temp_data])

# Save the data as a csv file for future use
tiingo_data.to_csv('tiingo_data.csv')

In [51]:
# Extract the list of symbols to use for quandl, limited because of some empty datasets
symbols = list(symbols_table[symbols_table['Date first added'].isnull() |
             (pd.to_datetime(symbols_table['Date first added']) < dt.datetime(2018, 3, 27))]['Symbol'])

#set the time frame to fetch quandl stock data
start = dt.datetime(2008, 3, 27)
end = dt.datetime(2019, 4, 19)
apikey='D9qsQxNxssmZNUxY5bBB'

quandl_data_1 = pd.DataFrame()
# Get stockdata using pandas_datareader and combine in a single dataframe
for stock in tqdm_notebook(symbols):
    # Initialise the timer
    time.sleep(0.25)
    # Extract stock data and stack it in a single dataframe
    if '.' in stock:
        stock = stock.replace('.', '_')
        temp_content = requests.get(f'https://www.quandl.com/api/v3/datasets/WIKI/{stock}.csv?start_date={start}&end_date={end}&order=asc&api_key={apikey}').content
        temp_data = pd.read_csv(io.StringIO(temp_content.decode('utf-8')))
        temp_data['symbol'] = stock
    elif stock == 'CPRI' or stock == 'JEF'  or stock == 'LIN'  or stock == 'RHT':
        print('Had to skip stock:' + stock)
    else:
        temp_data = web.DataReader(stock, "quandl", start, end, access_key = apikey)
        temp_data['symbol'] = stock
        quandl_data_1 = pd.concat([quandl_data_1, temp_data])

# Save the data as a csv file for future use
quandl_data_1.to_csv('quandl_data_1.csv')

Had to skip stock:CPRI
Had to skip stock:JEF
Had to skip stock:LIN
Had to skip stock:RHT


In [82]:
# This is another method that uses quandl built in python module

# Extract the list of symbols to use for quandl, limited because of some empty datasets
symbols = list(symbols_table[symbols_table['Date first added'].isnull() |
             (pd.to_datetime(symbols_table['Date first added']) < dt.datetime(2018, 3, 27))]['Symbol'])
symbols = [w.replace('.', '_') for w in symbols]
symbols_part1 = symbols[:240]
symbols_part2 = symbols[240:]

#set the time frame to fetch quandl stock data
start = '2008-3-27'
end = '2019-4-19'

# Get stockdata using pandas_datareader and combine in a single dataframe
quandl.ApiConfig.api_key = 'D9qsQxNxssmZNUxY5bBB'
quandl_data_2 = quandl.get_table('WIKI/PRICES', ticker = symbols_part1,
                        date = { 'gte': start, 'lte': end }, paginate=True)

quandl_temp = quandl.get_table('WIKI/PRICES', ticker = symbols_part2,
                        date = { 'gte': start, 'lte': end }, paginate=True)

quandl_data_2 = pd.concat([quandl_data_2, quandl_temp])

# Save the data as a csv file for future use
quandl_data_2.to_csv('quandl_data_2.csv')

In [ ]:
# Reload the data straight from CSV if needed
iex_data = pd.read_csv('iex_data.csv')
quandl_data_1 = pd.read_csv('quandl_data_1.csv')
quandl_data_2 = pd.read_csv('quandl_data_2.csv')
tiingo_data = pd.read_csv('tiingo_data.csv')

Rest of this notebook will only work with tiingo_data, since it is the most recent, complete and reliable datasource that I was able to obtain. The processed return data frames are the used for all future project predictions and testing.

In [113]:
tiingo_data.head()

,symbol,date,adjClose,adjHigh,adjLow,adjOpen,adjVolume,close,divCash,high,low,open,splitFactor,volume
0,MMM,2009-04-20,40.270745,41.456319,40.007285,41.371081,4697100,51.97,0.0,53.50,51.63,53.39,1.0,4697100
1,MMM,2009-04-21,41.293593,41.417574,40.293992,40.495462,5059100,53.29,0.0,53.45,52.00,52.26,1.0,5059100
2,MMM,2009-04-22,41.115369,42.114970,40.487713,40.774420,5893700,53.06,0.0,54.35,52.25,52.62,1.0,5893700
3,MMM,2009-04-23,42.006487,42.099473,40.456718,41.254849,6210900,54.21,0.0,54.33,52.21,53.24,1.0,6210900
4,MMM,2009-04-24,44.168414,44.183912,41.642291,42.045231,11541000,57.00,0.0,57.02,53.74,54.26,1.0,11541000


In [119]:
# Get list of all stocks
org_stocklist = tiingo_data['symbol'].unique()

# Get list of stocks with data in all time periods
filled_limit = max(tiingo_data['symbol'].value_counts())
lim_stocklist = tiingo_data.groupby("symbol").filter(lambda x: len(x) == full_limit)['symbol'].unique()

In [126]:
org_returns = pd.DataFrame()

for stock in org_stocklist:
    current = tiingo_data[tiingo_data['symbol'] == stock].sort_values('date')
    current['prev_adjClose'] = current['adjClose'].shift(1)
    current[stock] = (current['adjClose'] - current['prev_adjClose']) / current['prev_adjClose']
    current = current.set_index('date')[stock]
    
    org_returns = org_returns.join(current.to_frame(), how='outer')
org_returns = org_returns[1:]
org_returns['index'] = org_returns.mean(axis=1)

org_returns.to_csv('org_returns.csv')

In [127]:
lim_returns = pd.DataFrame()

for stock in lim_stocklist:
    current = tiingo_data[tiingo_data['symbol'] == stock].sort_values('date')
    current['prev_adjClose'] = current['adjClose'].shift(1)
    current[stock] = (current['adjClose'] - current['prev_adjClose']) / current['prev_adjClose']
    current = current.set_index('date')[stock]
    
    lim_returns = lim_returns.join(current.to_frame(), how='outer')
lim_returns = lim_returns[1:]
lim_returns['index'] = lim_returns.mean(axis=1)

lim_returns.to_csv('lim_returns.csv')